## dataset preparation

In [1]:
import cv2
import numpy as np
from keras.models import load_model
import pandas as pd
from tqdm import tqdm

In [2]:
def get_data_from_local(file):
    face = cv2.imread(file,cv2.IMREAD_COLOR)
    face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    face =cv2.resize(face, (200,200))
    face = np.asarray(face)
    face = face.astype('float32')
    face /= 255
    return np.array(face)

In [23]:
data1 = pd.read_csv('celebA/list_attr_celeba.csv')[:5000]
data1 = data1.replace(-1,0)
data1.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,0,1,1,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,0,1
1,000002.jpg,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
2,000003.jpg,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
3,000004.jpg,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
4,000005.jpg,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1


In [24]:
data = {
    'age':[],
    'gender':[],
    'race':[]
}
age_model = load_model('model_3/final_cnn_age_model_checkpoint.tf')
gender_model = load_model('model_3/final_cnn_gender_model_checkpoint.tf')
race_model = load_model('model_3/final_cnn_race_model_checkpoint.tf')
for image in tqdm(data1['image_id']):
    temp = image
    image = get_data_from_local('celebA/img_align_celeba/img_align_celeba/'+image)
    image = np.expand_dims(image, axis=0)
    data['age'].append(age_model.predict(image).argmax())
    data['gender'].append(gender_model.predict(image).argmax())
    data['race'].append(race_model.predict(image).argmax())

100%|██████████| 5000/5000 [15:38<00:00,  5.33it/s]


In [26]:
data1.columns

Index(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')

In [27]:
data1['age'] = data['age']
data1['gender'] = data['gender']
data1['race'] = data['race'] 

In [28]:
# data1 = data1.drop(columns=['Wearing_Lipstick','Wavy_Hair','Wearing_Earrings','Wearing_Hat','Wearing_Necktie','Young','Mustache','High_Cheekbones','Mouth_Slightly_Open','Goatee'])
# len(data1.columns)

In [29]:
data2 = pd.get_dummies(data=data1,columns=['age','gender','race'])

In [30]:
print(data2.columns)
len(data2.columns)

Index(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young', 'age_0', 'age_3', 'age_4', 'gender_0',
       'gender_1', 'race_0', 'race_1'],
      dtype='object')


48

In [31]:
data2['age_1'] = np.zeros(len(data1))
data2['age_2'] = np.zeros(len(data1))
data2['age_5'] = np.zeros(len(data1))
data2['age_6'] = np.zeros(len(data1))
data2['age_7'] = np.zeros(len(data1))
data2['race_2'] = np.zeros(len(data1))
data2['race_3'] = np.zeros(len(data1))
data2['race_4'] = np.zeros(len(data1))

In [32]:
len(data2.columns)

56

In [33]:
data2.to_csv('decode_datasest.csv')

# prediction

In [34]:
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping 
from keras.models import load_model

In [35]:
def training_dataset_(
    file_path,
    image_per_batch,
    start_row,
    ):
    df = pd.read_csv(file_path)[start_row:start_row+image_per_batch]
    df = df.drop('Unnamed: 0',axis=1)

    y = get_data_from_local(df)
    x = df.drop('image_id',axis=1)
    
    x = x.astype('float64')
        
    return x.values , y

In [36]:
def get_data_from_local(X_train):
    X_data =[]
    for file in tqdm(X_train['image_id']):
        file = 'celebA/img_align_celeba/img_align_celeba/' + file
        face = cv2.imread(file,cv2.IMREAD_COLOR)
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face =cv2.resize(face, (150,150))
        face = np.asarray(face)
        face = face.astype('float32')
        face /= 255
        X_data.append(face)
    return np.array(X_data)

In [37]:
c_a_e = load_model('decoder_cnn_50.tf/')
c_a_e.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [38]:
c_a_e.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 900)               27900     
_________________________________________________________________
reshape (Reshape)            (None, 10, 10, 9)         0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 20, 20, 9)         0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 22, 22, 9)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 22, 22, 16)        1312      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 24, 24, 16)       

In [39]:
def training(
    start_row,
    number_of_labels,
    file_path,
    image_per_batch,
    max_limit,
    path_of_model
):

    checkpoint = ModelCheckpoint(filepath=path_of_model,
                             monitor='val_accuracy',
                             save_format='tf',
                             save_best_only=True,
                             save_weights_only=False,
                             verbose=1
                            )
    early_stopping_monitor = EarlyStopping(patience=4,verbose=1) 
    
    
    for x in range(start_row,max_limit,image_per_batch):
        
        print()
        print('**********{} -- {} out of {}************'.format(x,x+image_per_batch,max_limit))
        print()
    
        X_train , y_train   =   training_dataset_(
                        file_path = file_path,
                        start_row = x,
                        image_per_batch = image_per_batch
        )
                                    
        F_mnist = c_a_e.fit(X_train,y_train,
                            shuffle=True,
                            epochs=30,
                            validation_split=0.2,
                            batch_size=64,
                            callbacks=[checkpoint,early_stopping_monitor]
       )


In [ ]:
training(
    start_row = 0,
    number_of_labels = 3,
    file_path = 'decode_datasest.csv',
    image_per_batch = 500,
    max_limit = 500,
    path_of_model = 'final_dec_cnn_checkpoint.tf'
)

In [ ]:
# 